In [6]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install pandas numpy


     -------------------------------------- 10.6/10.6 MB 502.0 kB/s eta 0:00:00
     -------------------------------------- 15.5/15.5 MB 469.3 kB/s eta 0:00:00
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import zipfile
import numpy as np
import pandas as pd


In [13]:
#path = '/content/archive.zip'
#from google.colab import drive
#drive.mount('/content/drive/')

path=r'D:\Youtube analysis\archive.zip'

In [10]:
def file_extract(path_of_file):
  csv_files=[]
  json_files=[]

  with zipfile.ZipFile(path,mode="r") as archive:
    for filename in archive.namelist():
      file_ext = filename[filename.index(".")+1:]
      if file_ext == "csv":
        df = pd.read_csv(archive.open(filename),encoding = "latin-1")
        df["region"]=filename[0:2]
        csv_files.append(df);
      elif file_ext == "json":
        df_region = pd.read_json(archive.open(filename))
        df_id = pd.read_json((df_region)['items'].to_json(), orient='index')['id']
        df_category_title = pd.read_json(pd.read_json(df_region['items'].to_json(), orient='index')['snippet'].to_json(), orient='index')['title']
        data = {'category_id' : df_id, 'category_title': df_category_title}
        df = pd.concat(data,axis=1)
        df["region"]=filename[0:2]
        json_files.append(df)
  return pd.concat(csv_files,ignore_index=True),pd.concat(json_files,ignore_index=True)


In [15]:
df_region,df_category = file_extract(path)

In [16]:
df_region.head()
df_category.head()


,category_id,category_title,region
0,1,Film & Animation,CA
1,2,Autos & Vehicles,CA
2,10,Music,CA
3,15,Pets & Animals,CA
4,17,Sports,CA


In [17]:
combined_df = pd.merge(df_region,df_category,on=["region","category_id"],how="left")

In [18]:
combined_df_without_null = combined_df[~pd.isnull(combined_df['category_title'])]

In [19]:
combined_df_without_null.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,region,category_title
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA,Music
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,Comedy
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA,Comedy
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,Entertainment
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA,Music


In [20]:
columns_drop = ["title","tags","thumbnail_link","description"]

In [21]:
combine_df_clean = combined_df_without_null.drop(columns=columns_drop)

In [24]:
#import os
#os.makedirs('D/Youtube analysis', exist_ok=True)
combine_df_clean.to_csv(r'D:\Youtube analysis\clean.csv',index = False)

In [26]:
clean_df = pd.read_csv(r'D:\Youtube analysis\clean.csv')
clean_df.head()

,video_id,trending_date,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,region,category_title
0,n1WpP7iowLc,17.14.11,EminemVEVO,10,2017-11-10T17:00:03.000Z,17158579,787425,43420,125882,False,False,False,CA,Music
1,0dBIkQ4Mz1M,17.14.11,iDubbbzTV,23,2017-11-13T17:00:00.000Z,1014651,127794,1688,13030,False,False,False,CA,Comedy
2,5qpjK5DgCt4,17.14.11,Rudy Mancuso,23,2017-11-12T19:05:24.000Z,3191434,146035,5339,8181,False,False,False,CA,Comedy
3,d380meD0W0M,17.14.11,nigahiga,24,2017-11-12T18:01:41.000Z,2095828,132239,1989,17518,False,False,False,CA,Entertainment
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran,10,2017-11-09T11:04:14.000Z,33523622,1634130,21082,85067,False,False,False,CA,Music


In [28]:
import mysql.connector


# Configuration for MySQL database
server = 'localhost'
database = 'YouTubeAPI'
username = 'root'
password = ''

# Establish the connection
cnxn = mysql.connector.connect(
    host=server,
    database=database,
    user=username,
    password=password
)

# Create a cursor object
cursor = cnxn.cursor()

# Assuming 'df' is a pandas DataFrame containing your data
for index, row in clean_df.iterrows():
    query = '''INSERT INTO youtube_stats(video_id, trending_date, channel_title, category_id, publish_time, views, likes, dislikes, comment_count, comments_disabled, ratings_disabled, video_error_or_removed, region, category_title) 
               VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    data = (row.video_id, row.trending_date, row.channel_title, row.category_id, row.publish_time, row.views, row.likes, row.dislikes, row.comment_count, row.comments_disabled, row.ratings_disabled, row.video_error_or_removed, row.region, row.category_title)
    cursor.execute(query, data)

# Commit the transaction
cnxn.commit()

# Close the cursor and the connection
cursor.close()
cnxn.close()
